# Table of Contents

### 01. Import Libraries & Data
### 02. Clean Data
### 03. Merge Data
### 04. Group Data
### 05. Export Data

# 01. Import Libraries & Data

In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd

In [2]:
# Define path

path = r'C:\Users\chery\Desktop\Achievement 6\04-2021 PPP Loan Analysis\02 Data\Prepared Data'

In [3]:
# Import ppp data

df = pd.read_pickle(os.path.join(path, 'Task 6.3 Merged.pkl'))

In [10]:
df.head()

STATE_NAME STATE_FIPS STATE_ABBR  \
0  Washington         53         WA   
1  Washington         53         WA   
2  Washington         53         WA   
3  Washington         53         WA   
4  Washington         53         WA   

                                            geometry  loan_amount  \
0  MULTIPOLYGON (((-122.40202 48.22522, -122.4628...     149992.0   
1  MULTIPOLYGON (((-122.40202 48.22522, -122.4628...     149980.0   
2  MULTIPOLYGON (((-122.40202 48.22522, -122.4628...     149975.0   
3  MULTIPOLYGON (((-122.40202 48.22522, -122.4628...     149950.0   
4  MULTIPOLYGON (((-122.40202 48.22522, -122.4628...     149922.0   

             city      zip industry_code                    business_type  \
0      Bellingham  98225.0           nan         Subchapter S Corporation   
1  Spokane Valley  99216.0      621210.0  Limited  Liability Company(LLC)   
2        Kirkland  98033.0      339999.0         Subchapter S Corporation   
3    Port Orchard  98367.0      238210.0  Limited  Liability Company(LLC)   
4      Burlington  98233.0      999990.0         Subchapter S Corporation   

  business_owner_race business_owner_gender business_owner_vet_status  \
0                 NaN                   NaN                       NaN   
1                 NaN                   NaN                       NaN   
2                 NaN                   NaN                       NaN   
3                 NaN                   NaN                       NaN   
4                 NaN                   NaN                       NaN   

   jobs_reported date_loan_approved  \
0           11.0         05/01/2020   
1           12.0         04/29/2020   
2            6.0         04/16/2020   
3           10.0         06/13/2020   
4           12.0         05/03/2020   

                                      lender congressional_district _merge  
0  JPMorgan Chase Bank, National Association                  WA-02   both  
1                                Banner Bank                  WA-05   both  
2                               Seattle Bank                  WA-01   both  
3                              Kabbage, Inc.                  WA-06   both  
4                           Cross River Bank                  WA-02   both

In [8]:
# Import industry data

df_industry = pd.read_pickle(os.path.join(path, 'industry_cleaned.pkl'))

In [9]:
df_industry.head()

industry_code                                 industry_title
0        111110                   NAICS 111110 Soybean farming
1        111120  NAICS 111120 Oilseed, except soybean, farming
2        111130          NAICS 111130 Dry pea and bean farming
3        111140                     NAICS 111140 Wheat farming
4        111150                      NAICS 111150 Corn farming

# 02. Clean Data

#### Drop Unnecessary Columns

In [30]:
#Drop Unneccessary Columns

df_ppp = df.drop(columns = ['STATE_FIPS','STATE_ABBR','geometry', 'city','zip','business_owner_race','business_owner_gender','business_owner_vet_status','congressional_district','_merge'])

#### Ensure Common Column for Merge

In [32]:
#Drop decimal & zero from industry code
df_ppp['industry_code'] = df_ppp['industry_code'].astype(str).replace('\.0', '', regex=True)

In [33]:
df_ppp.head()

STATE_NAME  loan_amount industry_code                    business_type  \
0  Washington     149992.0           nan         Subchapter S Corporation   
1  Washington     149980.0        621210  Limited  Liability Company(LLC)   
2  Washington     149975.0        339999         Subchapter S Corporation   
3  Washington     149950.0        238210  Limited  Liability Company(LLC)   
4  Washington     149922.0        999990         Subchapter S Corporation   

   jobs_reported date_loan_approved                                     lender  
0           11.0         05/01/2020  JPMorgan Chase Bank, National Association  
1           12.0         04/29/2020                                Banner Bank  
2            6.0         04/16/2020                               Seattle Bank  
3           10.0         06/13/2020                              Kabbage, Inc.  
4           12.0         05/03/2020                           Cross River Bank

# 03. Merge Data

In [36]:
df_merged = df_ppp.merge(df_industry, on = 'industry_code')

In [37]:
df_merged.head()

STATE_NAME  loan_amount industry_code                    business_type  \
0  Washington     149980.0        621210  Limited  Liability Company(LLC)   
1  Washington     149900.0        621210                      Corporation   
2  Washington     149900.0        621210         Subchapter S Corporation   
3  Washington     149540.0        621210  Limited  Liability Company(LLC)   
4  Washington     149500.0        621210  Limited  Liability Company(LLC)   

   jobs_reported date_loan_approved                               lender  \
0           12.0         04/29/2020                          Banner Bank   
1           17.0         04/06/2020                          Umpqua Bank   
2            0.0         04/14/2020                Washington Trust Bank   
3           12.0         04/15/2020                 Cashmere Valley Bank   
4           12.0         04/30/2020  First-Citizens Bank & Trust Company   

                     industry_title  
0  NAICS 621210 Offices of dentists  
1  NAICS 621210 Offices of dentists  
2  NAICS 621210 Offices of dentists  
3  NAICS 621210 Offices of dentists  
4  NAICS 621210 Offices of dentists

# 04. Group Data

In [52]:
#Group by State + Loan Amount & Loan Count

df_state = df_merged.groupby('STATE_NAME').agg({'loan_amount': ['sum','mean','count']})

In [53]:
df_state

loan_amount                      
                               sum          mean   count
STATE_NAME                                              
Alabama               1.714225e+09  31082.387444   55151
Alaska                3.272528e+08  35520.768091    9213
Arizona               2.014685e+09  32990.840483   61068
Arkansas              9.647876e+08  29170.575157   33074
California            1.507472e+10  34300.795698  439486
Colorado              2.517731e+09  32200.166220   78190
Connecticut           1.570339e+09  33728.665016   46558
Delaware              3.207053e+08  36530.955996    8779
District of Columbia  3.555794e+08  37900.167378    9382
Florida               9.548429e+09  28907.214322  330313
Georgia               4.040994e+09  30061.100854  134426
Hawaii                5.716705e+08  31394.943235   18209
Idaho                 7.490164e+08  31787.820598   23563
Illinois              5.071050e+09  30242.786202  167678
Indiana               2.039002e+09  33393.969230   61059
Iowa                  1.244850e+09  26228.344898   47462
Kansas                1.197920e+09  28211.569862   42462
Kentucky              1.294728e+09  33715.106689   38402
Louisiana             1.867165e+09  30627.332802   60964
Maine                 6.463414e+08  29029.483327   22265
Maryland              2.220183e+09  34437.995257   64469
Massachusetts         2.818150e+09  36255.156246   77731
Michigan              3.370687e+09  37533.398620   89805
Minnesota             2.349528e+09  30864.886166   76123
Mississippi           1.087490e+09  26760.420704   40638
Missouri              2.212408e+09  30618.176453   72258
Montana               5.790845e+08  30058.890558   19265
Nebraska              8.432426e+08  26181.152684   32208
Nevada                1.015466e+09  31345.415303   32396
New Hampshire         5.853791e+08  36883.566308   15871
New Jersey            3.754367e+09  33986.016491  110468
New Mexico            5.933634e+08  34403.863343   17247
New York              8.237915e+09  32894.422038  250435
North Carolina        3.320264e+09  33683.980814   98571
North Dakota          4.611814e+08  28355.964137   16264
Ohio                  3.666299e+09  35094.608233  104469
Oklahoma              1.495809e+09  28362.475213   52739
Oregon                1.606836e+09  34574.201424   46475
Pennsylvania          4.109643e+09  34867.969007  117863
Rhode Island          3.799542e+08  36334.910435   10457
South Carolina        1.674623e+09  32413.734292   51664
South Dakota          4.727727e+08  26707.302548   17702
Tennessee             2.301293e+09  31139.791065   73902
Texas                 1.022906e+10  32286.152898  316825
Utah                  1.271631e+09  32704.042701   38883
Vermont               3.166462e+08  34731.408016    9117
Virginia              2.931154e+09  34178.969355   85759
Washington            2.723766e+09  35292.003362   77178
West Virginia         4.808478e+08  33746.073719   14249
Wisconsin             2.201705e+09  32452.939957   67843
Wyoming               3.333379e+08  31135.612729   10706

In [55]:
#Group by Industry + Loan Amount & Loan Count

df_industry = df_merged.groupby('industry_title').agg({'loan_amount': ['sum','mean','count']})

In [63]:
pd.set_option('display.max_rows', None)

In [64]:
df_industry

loan_amount  \
                                                             sum   
industry_title                                                     
NAICS 111110 Soybean farming                        1.495085e+08   
NAICS 111120 Oilseed, except soybean, farming       6.284313e+06   
NAICS 111130 Dry pea and bean farming               5.035359e+06   
NAICS 111140 Wheat farming                          5.273634e+07   
NAICS 111150 Corn farming                           3.149234e+08   
NAICS 111160 Rice farming                           1.684883e+07   
NAICS 111191 Oilseed and grain combination farming  1.096607e+08   
NAICS 111199 All other grain farming                1.397508e+08   
NAICS 111211 Potato farming                         5.179087e+07   
NAICS 111219 Other vegetable and melon farming      6.320870e+07   
NAICS 111310 Orange groves                          6.797501e+06   
NAICS 111320 Citrus, except orange, groves          4.317271e+06   
NAICS 111331 Apple orchards                         3.603037e+07   
NAICS 111332 Grape vineyards                        2.765698e+07   
NAICS 111333 Strawberry farming                     6.523700e+06   
NAICS 111334 Berry, except strawberry, farming      1.657793e+07   
NAICS 111335 Tree nut farming                       3.395536e+07   
NAICS 111336 Fruit and tree nut combination far...  1.818673e+07   
NAICS 111339 Other noncitrus fruit farming          2.780754e+07   
NAICS 111411 Mushroom production                    3.882440e+06   
NAICS 111419 Other food crops grown under cover     1.290417e+07   
NAICS 111421 Nursery and tree production            9.606430e+07   
NAICS 111422 Floriculture production                3.284414e+07   
NAICS 111910 Tobacco farming                        1.388535e+07   
NAICS 111920 Cotton farming                         5.945786e+07   
NAICS 111930 Sugarcane farming                      8.492561e+06   
NAICS 111940 Hay farming                            3.070787e+07   
NAICS 111991 Sugar beet farming                     1.227936e+07   
NAICS 111992 Peanut farming                         6.123464e+06   
NAICS 111998 All other miscellaneous crop farming   2.935528e+08   
NAICS 112111 Beef cattle ranching and farming       2.261704e+08   
NAICS 112112 Cattle feedlots                        4.794511e+07   
NAICS 112120 Dairy cattle and milk production       2.667408e+08   
NAICS 112210 Hog and pig farming                    6.993176e+07   
NAICS 112310 Chicken egg production                 1.110753e+07   
NAICS 112320 Broilers and meat type chicken pro...  1.210335e+07   
NAICS 112330 Turkey production                      7.449631e+06   
NAICS 112340 Poultry hatcheries                     4.124152e+06   
NAICS 112390 Other poultry production               8.927565e+06   
NAICS 112410 Sheep farming                          4.551198e+06   
NAICS 112420 Goat farming                           2.569951e+06   
NAICS 112511 Finfish farming and fish hatcheries    1.099582e+07   
NAICS 112512 Shellfish farming                      7.123447e+06   
NAICS 112519 Other aquaculture                      1.173711e+07   
NAICS 112910 Apiculture                             1.965206e+07   
NAICS 112920 Horses and other equine production     4.723790e+07   
NAICS 112930 Fur-bearing animal and rabbit prod...  2.715265e+06   
NAICS 112990 All other animal production            5.303458e+07   
NAICS 113110 Timber tract operations                4.648766e+07   
NAICS 113210 Forest nursery and gathering fores...  1.042479e+07   
NAICS 113310 Logging                                1.788366e+08   
NAICS 114111 Finfish fishing                        4.986368e+07   
NAICS 114112 Shellfish fishing                      3.558029e+07   
NAICS 114119 Other marine fishing                   2.028682e+07   
NAICS 114210 Hunting and trapping                   1.567475e+07   
NAICS 115111 Cotton ginning                         1.857270e+07   
NAICS 115112 Soil preparation, planting, 

In [57]:
#Group by Industry + Loan Amount & Loan Count

df_lender = df_merged.groupby('lender').agg({'loan_amount': ['sum','mean','count']})

In [65]:
df_lender

loan_amount  \
                                                             sum   
lender                                                             
121 Financial CU                                    1.297636e+07   
1880 Bank                                           9.079910e+06   
1st Advantage Bank                                  4.679084e+06   
1st Advantage FCU                                   1.535184e+06   
1st Bank                                            3.886842e+05   
1st Bank Yuma                                       1.195790e+07   
1st Bank in Hominy                                  6.213620e+05   
1st Bank of Sea Isle City                           1.951415e+06   
1st Bergen FCU                                      8.489400e+04   
1st Capital Bank                                    1.550390e+07   
1st Choice CU                                       9.630403e+06   
1st Colonial Community Bank                         1.069028e+07   
1st Community Bank                                  7.998699e+05   
1st Community CU                                    4.207738e+05   
1st Community FCU                                   1.612900e+06   
1st Constitution Bank                               1.635024e+07   
1st Equity Bank                                     3.020849e+06   
1st Financial Bank USA                              2.747203e+06   
1st Financial FCU                                   1.981000e+05   
1st Liberty FCU                                     1.422524e+05   
1st MidAmerica CU                                   2.444275e+06   
1st Mississippi FCU                                 1.192460e+05   
1st National Bank                                   4.673984e+06   
1st Security Bank of Washington                     1.669790e+07   
1st Source Bank                                     1.020486e+08   
1st State Bank                                      1.651007e+07   
1st Summit Bank                                     1.693798e+07   
1st Trust Bank, Inc.                                2.442443e+06   
1st United Bank                                     3.885650e+06   
20th Century Fox FCU                                5.064500e+04   
21st Century Bank                                   2.220848e+07   
22nd State Bank                                     3.066471e+06   
360 FCU                                             5.739340e+05   
4Front CU                                           5.356877e+06   
5Star Bank                                          6.066256e+06   
A N E C A FCU                                       3.433933e+06   
A O D FCU                                           4.179523e+05   
A+ FCU                                              7.869430e+06   
AB&T                                                5.953155e+06   
ABNB FCU                                            7.234084e+06   
ACB Bank                                            1.710775e+06   
ACMG FCU                                            6.266381e+05   
ACNB Bank                                           2.117101e+07   
ACT 1st FCU                                         4.266743e+05   
AEA FCU                                             1.150200e+06   
AG CREDIT ACA                                       6.411583e+06   
ALPS FCU                                            1.178556e+06   
AMG National Trust Bank                             1.820982e+06   
AMPAC Tri-State CDC, Inc.                           2.982920e+06   
ANB Bank                                            6.975347e+07   
ARC FCU                                             3.508000e+05   
AVB Bank                                            7.780678e+06   
Abacus Federal Savings Bank                         3.227779e+06   
Abbeville First Bank, SSB                           1.631526e+06   
Abbey CU Inc                                        6.216121e+05   
AbbyBank                                            1.039330e+07   
Abington Bank                            

In [59]:
#Group by Date + Loan Amount & Loan Count

df_date = df_merged.groupby('date_loan_approved').agg({'loan_amount': ['sum','mean','count']})

In [66]:
df_date

loan_amount                      
                             sum          mean   count
date_loan_approved                                    
04/03/2020          7.939953e+08  60055.615735   13221
04/04/2020          1.531502e+09  57075.326806   26833
04/05/2020          1.679440e+09  57917.715445   28997
04/06/2020          2.760286e+09  54312.820837   50822
04/07/2020          3.897102e+09  53157.044451   73313
04/08/2020          3.709796e+09  51449.200593   72106
04/09/2020          4.018299e+09  50345.162345   79815
04/10/2020          4.374083e+09  48051.529201   91029
04/11/2020          3.324682e+09  49312.985942   67420
04/12/2020          1.526305e+09  50704.439244   30102
04/13/2020          4.754632e+09  46019.845094  103317
04/14/2020          6.936823e+09  44651.719751  155354
04/15/2020          8.938003e+09  40292.309528  221829
04/16/2020          2.110073e+09  39184.272877   53850
04/27/2020          7.469269e+09  36027.383259  207322
04/28/2020          1.118886e+10  32884.045024  340252
04/29/2020          6.131181e+09  30033.609321  204144
04/30/2020          7.281553e+09  28686.842666  253829
05/01/2020          7.156971e+09  32311.089444  221502
05/02/2020          8.849092e+08  24621.847145   35940
05/03/2020          9.522101e+09  36416.592984  261477
05/04/2020          1.043910e+09  21198.302069   49245
05/05/2020          1.040485e+09  21598.017756   48175
05/06/2020          9.911255e+08  20955.357557   47297
05/07/2020          8.390048e+08  20353.324999   41222
05/08/2020          7.899748e+08  19804.822985   39888
05/09/2020          1.623191e+08  19078.405731    8508
05/10/2020          6.551784e+07  18810.750057    3483
05/11/2020          6.117679e+08  20358.332560   30050
05/12/2020          5.393309e+08  19768.743468   27282
05/13/2020          5.274710e+08  20052.881006   26304
05/14/2020          4.997641e+08  20248.938542   24681
05/15/2020          4.487703e+08  19612.370963   22882
05/16/2020          8.229673e+07  17498.772165    4703
05/17/2020          6.343884e+07  18652.994804    3401
05/18/2020          3.975520e+08  20092.590250   19786
05/19/2020          3.933057e+08  19731.384760   19933
05/20/2020          3.810405e+08  19403.223545   19638
05/21/2020          3.711689e+08  19780.906803   18764
05/22/2020          2.847793e+08  20795.916146   13694
05/23/2020          5.156507e+07  18575.312622    2776
05/24/2020          3.477468e+07  18244.849627    1906
05/25/2020          2.564240e+07  17072.169993    1502
05/26/2020          2.884826e+08  19821.530417   14554
05/27/2020          2.865282e+08  19471.845375   14715
05/28/2020          2.545405e+08  19899.972511   12791
05/29/2020          2.386667e+08  19046.102799   12531
05/30/2020          4.053500e+07  17662.307207    2295
05/31/2020          2.525867e+07  17288.620137    1461
06/01/2020          2.036252e+08  19442.873058   10473
06/02/2020          2.083025e+08  19612.326192   10621
06/03/2020          2.298427e+08  19718.828207   11656
06/04/2020          1.846437e+08  19722.681881    9362
06/05/2020          2.199846e+08  20431.377001   10767
06/06/2020          2.713692e+07  20403.700331    1330
06/07/2020          1.731198e+07  17847.403680     970
06/08/2020          1.560422e+08  20656.901604    7554
06/09/2020          1.387396e+08  20378.905025    6808
06/10/2020          1.510722e+08  20073.375961    7526
06/11/2020          1.623698e+08  20485.715093    7926
06/12/2020          2.147939e+08  18655.020278   11514
06/13/2020          8.920158e+07  16614.189018    5369
06/14/2020          2.400480e+07  17715.719410    1355
06/15/2020          2.075677e+08  18547.736141   11191
06/16/2020          2.653309e+08  17117.020930   15501
06/17/2020          2.597114e+08  19330.959671   13435
06/18/2020          2.939284e+08  19633.185754   14971
06/19/2020          2.274598e+08  19844.690014   11462
06/20/2020          5.268645e+07  18924.729328    2784
06/21/2020          2.813409e+07  18151.027755    1550
06/22/

# 04. Export Data

In [67]:
# Export data

df_merged.to_pickle(os.path.join(path, 'ppp_industry_merged_final.pkl'))